# Introduction to PyTorch part 2

## Class-based Dataset Definition

In [1]:
from torch.utils.data import Dataset
import pandas as pd


class WaterDataset(Dataset):
    def __init__(self, csv_path: str):
        super().__init__()
        df = pd.read_csv(csv_path)
        self.data = df.to_numpy()

    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self, idx: int):
        features = self.data[idx, :-1]
        label = self.data[idx, -1]
        return features, label

In [2]:
from torch.utils.data import DataLoader

dataset_train = WaterDataset("./data/water_dataset.csv")
dataloader_train = DataLoader(dataset_train, batch_size=2, shuffle=True)

In [3]:
features, labels = next(iter(dataloader_train))
print(f"Features: {features}")
print(f"Labels: {labels}")

Features: tensor([[0.4550, 0.5399, 0.3472, 0.4221, 0.4812, 0.5093, 0.4017, 0.7009, 0.6860],
        [0.4128, 0.4548, 0.7215, 0.4815, 0.5765, 0.6557, 0.7948, 0.4355, 0.4857]],
       dtype=torch.float64)
Labels: tensor([0., 1.], dtype=torch.float64)


## Class-based Model Definition

### From this:

In [4]:
from torch import nn


net = nn.Sequential(
    nn.Linear(9, 16),
    nn.ReLU(),
    nn.Linear(16, 8),
    nn.ReLU(),
    nn.Linear(8, 1),
    nn.Sigmoid(),
)

### To this:

In [5]:
from torch import nn


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(9, 16)
        self.fc2 = nn.Linear(16, 8)
        self.fc3 = nn.Linear(8, 1)

    def forward(self, x):
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.relu(self.fc2(x))
        x = nn.functional.sigmoid(self.fc3(x))
        return x

In [6]:
net = Net()

## Taining Loop

In [7]:
from torch import nn, optim

criterion = nn.BCELoss()
optimizer = optim.SGD(net.parameters(), lr=0.01)

for epoch in range(10):  # iterate over epochs and training batches
    for features, labels in dataloader_train:
        features = features.float()  # convert features to float32
        labels = labels.float()  # convert labels to float32
        optimizer.zero_grad()  # clear gradients
        outputs = net(features)  # forward pass and get model's output
        loss = criterion(outputs, labels.view(-1, 1))  # compute loss
        loss.backward()  # compute gradients
        optimizer.step()  # update params

### Some options for optimizer

In [8]:
optimizer = optim.SGD(net.parameters(), lr=0.01)  # Stochastic Gradient Descent
optimizer = optim.Adagrad(net.parameters(), lr=0.01)  # Adaptive Gradient
optimizer = optim.RMSprop(net.parameters(), lr=0.01)  # Root Mean Square Propagation
optimizer = optim.Adam(net.parameters(), lr=0.01)  # Adaptive Moment Estimation

### Model Evaluation

In [9]:
import torch
from torchmetrics import Accuracy

acc = Accuracy(task="binary")  # setup accuracy metric


net.eval()  # set model to evaluation mode
with torch.no_grad():  # iterate over test data batches with no gradients
    # for features, labels in dataloader_test:
    for features, labels in dataloader_train:
        features = features.float()  # convert features to float32
        labels = labels.float()  # convert labels to float32
        outputs = net(features)  # pass data to model
        preds = (
            outputs >= 0.5
        ).float()  # compute predicted labels (based on 0.5 treshold)
        acc(preds, labels.view(-1, 1))  # update accuracy metric

accuracy = acc.compute()
print(f"Accuracy: {accuracy}")

Accuracy: 0.5988063812255859
